In [43]:
import requests
import folium
import pandas as pd
import numpy as np

In [39]:
!pip install folium

    100% |████████████████████████████████| 81kB 17.4MB/s 


In [6]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})

In [8]:
!pip install wikipedia

  Running setup.py bdist_wheel for wikipedia ... done
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
distributed 1.21.8 requires msgpack, which is not installed.


In [9]:
df = pd.read_html(str(My_table),header=0)[0]
df.columns = ['PostalCode' , 'Borough','Neighborhood']
df.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# __We are ignoring the cells with a borough that is Not assigned.__

In [10]:
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.head(5)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


# __Below is the output of shape__

In [30]:
df.shape[0]

103

# __Use GEOCODER to find Co-Ords__

In [25]:
!pip install geopy

In [24]:
!pip install msgpack 

    100% |████████████████████████████████| 317kB 20.3MB/s 


In [32]:
!pip install geocoder

    100% |████████████████████████████████| 102kB 18.9MB/s 


In [33]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [34]:
latitude

43.653226

In [35]:
longitude

-79.3831843

In [36]:
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head(5)
coord = pd.read_csv('http://cocl.us/Geospatial_data')
coord.columns = ['PostalCode' , 'Latitude','Longitude']
df1 = df.merge(coord, on='PostalCode', how='outer')
df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [41]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [45]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
    ).add_to(map_toronto)  
    
map_toronto